# Лабораторная работа #1

Подключим Google Drive и укажем путь к датасету

In [1]:
from google.colab import drive
drive.mount ('/content/drive')

data_file_path = '/content/drive/MyDrive/Colab Notebooks/Laptop_price.csv'

Mounted at /content/drive


Загрузим датасет в PD и выполним разбиение данных

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(data_file_path)

X = df.drop(columns=['Price']) # Удалим все столбци, кроме 'Price', чтобы избежать запоминания ответов
y = df['Price'] # столбец 'Price' выделим как ключевой (в нем содержатся ответы)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # разделим на трейн и тест выборки

Работа с пайплайном

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import joblib

# Разделяем числовые и категориальные признаки
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()

# Создаем трансформеры
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
# Объеденяем трансформеры
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])
# Создаем финальный пайплайн. Он состоит из предобработчика (который объеденили ранее) и модели XGBRegressor
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])
# Обучаем модель на train выборке и бекапим на диск
pipeline.fit(X_train, y_train)
joblib.dump(pipeline, '/content/drive/MyDrive/Colab Notebooks/laptop_price_model.pkl')

['/content/drive/MyDrive/Colab Notebooks/laptop_price_model.pkl']

Сгенерируем SSH-ключи

In [4]:
#!rm ~/.ssh
!mkdir -p /root/.ssh
!chmod 777 /root/.ssh


!ssh-keygen -t rsa -b 4096 -C "IT_Sem4_Lab1" -f ~/.ssh/labw_ssh_key
!cp ~/.ssh/labw_ssh_key /content/drive/MyDrive/LabWorks
!cp ~/.ssh/labw_ssh_key.pub /content/drive/MyDrive/LabWorks

%cd ~/.ssh/
!ls

Generating public/private rsa key pair.
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/labw_ssh_key
Your public key has been saved in /root/.ssh/labw_ssh_key.pub
The key fingerprint is:
SHA256:WeafGN9GiEzb+UYMD/zT0uQulS/r040Ey5kmu25W4tg IT_Sem4_Lab1
The key's randomart image is:
+---[RSA 4096]----+
|                 |
|           .     |
|          + +   .|
|         B +.O =.|
|        S *.+=O.=|
|          o=B*.=.|
|         +.*+.Bo+|
|        . E  oo+o|
|         +o. .o. |
+----[SHA256]-----+
/root/.ssh
labw_ssh_key  labw_ssh_key.pub


/root/.ssh


Восстановим ранее сгенерированный ключ

In [19]:
!rm ~/.ssh
!mkdir -p /root/.ssh
!chmod 700 /root/.ssh

!cp /content/drive/MyDrive/LabWorks/labw_ssh_key ~/.ssh
!cp /content/drive/MyDrive/LabWorks/labw_ssh_key.pub ~/.ssh

%cd ~/.ssh
!ls

rm: cannot remove '/root/.ssh': Is a directory
/root/.ssh
id_rsa	id_rsa.pub


In [5]:
#Вывод публичного ключа
!cat ~/.ssh/labw_ssh_key.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDXBNNtMuMt4yZtos9wc8+0Zqh7AEob7JmZe9BR/JcaHes7Wa3d6AH29SADHm0d4etEPlw7NB/SrtbRyz7ikeVJC7/VM/fIJSZDnZJEcV2aeVAXNU7zglbkDgzgn+Vmq4U5OSIkYZwD7lpZwOOCzMx1e2AcjGRowQ0h2aU+IIlWqti+zZPuMB62VCCt/6rHc9gSW5AA7dEcrOnUIYL6a7DK9YE5EfPXPINHOJJTcgkS9zD14gLHhE+WteU74Pxwqxl6R49mgTFp7XdrFWZ8CMzvaYleXWTAfdrKWvysvsNXNeufZOxZmx4oCC067aG7pdAwSnR0h/8XeBz4+d9HWP9Y1B6gix0LD4qKKrD9O4vQ1N+B0Epzu+azZH6uktZELRdJnVjFgOLQNwiUGCV/V854c/nfbxYF9KN6HYGj8cL165u+0O4qfjRW5x3oBjxdjlC3Fz3Ag+siVTA2BjMxYYoAyoMXde4BNTXM7rwb7guQp4OsP4yo2/ZqjKecNbLppJy43FbzaaT8mcxsBC2E4ScMZjiFAphaecu7hSK9Kci+8OR+4iRXtXct25zT79zIChaS5ROB8hBL5JZnB6nH+OKvgzIHxp07t2LYdq/0d2PEDMY+UMDAb4wptzccqZ2J4HD+P9AbksDZy1Zve57hPsbvtsivhfMpVDozKTPMxVWLNw== IT_Sem4_Lab1


Создаем config и known_hosts

In [6]:
%cd ~/.ssh

!touch config
!touch known_hosts

!ls

/root/.ssh
config	known_hosts  labw_ssh_key  labw_ssh_key.pub


In [34]:
!sed -i 'd' ~/.ssh/config # очищаем config перед заполнением

!echo "Host github.com" >> ~/.ssh/config
#!echo "  Hostname github.com" >> ~/.ssh/config
!echo "  User git" >> ~/.ssh/config
!echo "  IdentityFile ~/.ssh/labw_ssh_key" >> ~/.ssh/config
!echo "  PreferredAuthentications publickey" >> ~/.ssh/config

#!echo "  IdentitiesOnly yes" >> ~/.ssh/config

!cat ~/.ssh/config

Host github.com
  User git
  IdentityFile ~/.ssh/id_rsa
  PreferredAuthentications publickey


In [24]:
!sed -i 'd' ~/.ssh/known_hosts # очищаем known_hosts перед заполнением

!echo "github.com ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIOMqqnkVzrm0SdG6UOoqKLsabgH5C9okWi0dh2l9GKJl" >> ~/.ssh/known_hosts
!echo "github.com ecdsa-sha2-nistp256 AAAAE2VjZHNhLXNoYTItbmlzdHAyNTYAAAAIbmlzdHAyNTYAAABBBEmKSENjQEezOmxkZMy7opKgwFB9nkt5YRrYMjNuG5N87uRgg6CLrbo5wAdT/y6v0mKV0U2w0WZ2YB/++Tpockg=" >> ~/.ssh/known_hosts
!echo "github.com ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQCj7ndNxQowgcQnjshcLrqPEiiphnt+VTTvDP6mHBL9j1aNUkY4Ue1gvwnGLVlOhGeYrnZaMgRK6+PKCUXaDbC7qtbW8gIkhL7aGCsOr/C56SJMy/BCZfxd1nWzAOxSDPgVsmerOBYfNqltV9/hWCqBywINIR+5dIg6JTJ72pcEpEjcYgXkE2YEFXV1JHnsKgbLWNlhScqb2UmyRkQyytRLtL+38TGxkxCflmO+5Z8CSSNY7GidjMIZ7Q4zMjA2n1nGrlTDkzwDCsw+wqFPGQA179cnfGWOWRVruj16z6XyvxvjJwbz0wQZ75XK5tKSb7FNyeIEs4TT4jk+S4dhPeAUC5y+bDYirYgM4GC7uEnztnZyaVWQ7B381AK4Qdrwt51ZqExKbQpTUNn+EjqoTwvqNj4kqx5QUCI0ThS/YkOxJCXmPUWZbhjpCg56i+2aB6CmK2JGhn57K5mj0MNdBXA4/WnwH6XoPWJzK5Nyu2zB3nAZp+S5hpQs+p1vN1/wsjk=" >> ~/.ssh/known_hosts

!cat ~/.ssh/known_hosts

github.com ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIOMqqnkVzrm0SdG6UOoqKLsabgH5C9okWi0dh2l9GKJl
github.com ecdsa-sha2-nistp256 AAAAE2VjZHNhLXNoYTItbmlzdHAyNTYAAAAIbmlzdHAyNTYAAABBBEmKSENjQEezOmxkZMy7opKgwFB9nkt5YRrYMjNuG5N87uRgg6CLrbo5wAdT/y6v0mKV0U2w0WZ2YB/++Tpockg=
github.com ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQCj7ndNxQowgcQnjshcLrqPEiiphnt+VTTvDP6mHBL9j1aNUkY4Ue1gvwnGLVlOhGeYrnZaMgRK6+PKCUXaDbC7qtbW8gIkhL7aGCsOr/C56SJMy/BCZfxd1nWzAOxSDPgVsmerOBYfNqltV9/hWCqBywINIR+5dIg6JTJ72pcEpEjcYgXkE2YEFXV1JHnsKgbLWNlhScqb2UmyRkQyytRLtL+38TGxkxCflmO+5Z8CSSNY7GidjMIZ7Q4zMjA2n1nGrlTDkzwDCsw+wqFPGQA179cnfGWOWRVruj16z6XyvxvjJwbz0wQZ75XK5tKSb7FNyeIEs4TT4jk+S4dhPeAUC5y+bDYirYgM4GC7uEnztnZyaVWQ7B381AK4Qdrwt51ZqExKbQpTUNn+EjqoTwvqNj4kqx5QUCI0ThS/YkOxJCXmPUWZbhjpCg56i+2aB6CmK2JGhn57K5mj0MNdBXA4/WnwH6XoPWJzK5Nyu2zB3nAZp+S5hpQs+p1vN1/wsjk=


In [25]:
!git --version

git version 2.34.1


Подключим git репозиторий и сделаем первый коммит

In [26]:
%cd /content/drive/MyDrive/Colab\ Notebooks
!git init
!git add Sem4Lab1.ipynb
!git add laptop_price_model.pkl
!git add Laptop_price.csv

!git config --global user.email "145021797+morganuk@users.noreply.github.com"
!git config --global user.name "Morgan"

!git commit -m "Добавлен ML-пайплайн"

/content/drive/MyDrive/Colab Notebooks
Reinitialized existing Git repository in /content/drive/MyDrive/Colab Notebooks/.git/
[main 55b6727] Добавлен ML-пайплайн
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite Sem4Lab1.ipynb (92%)


In [27]:
!git remote add origin git@github.com:morganuk/it_labwork_4sem.git

error: remote origin already exists.


In [141]:
#!git config --global ssh.knownHostsFile /.ssh/known_hosts
#!git config --global ssh.strictHostKeyChecking yes

In [36]:
!git branch -M main
!git push -u origin main

git@github.com: Permission denied (publickey).
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


In [35]:
!ssh -i ~/.ssh/labw_ssh_key -T git@github.com

git@github.com: Permission denied (publickey).
